In [106]:
import io
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [107]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_districts_in_India")
s = BeautifulSoup(r.content, 'html.parser')

In [108]:
tables = s.find_all('table')
len(tables)

54

In [109]:
a = -1
tableData = []
for i in tables:
    a+=1
    print(a)
    htmlData = str(i)
    txtData = re.sub(r"\[.*?\]", "", htmlData)
    df = pd.read_html(io.StringIO(txtData))[0]
    print(df.head())
    tableData.append(df)

0
  Districts of India                               Districts of India.1
0                NaN                                                NaN
1           Category         second/third-level administrative division
2           Location              States and union territories of India
3             Number                       797 (as of 20 November 2023)
4        Populations  Greatest: Thane, Maharashtra—11,060,148 (2011 ...
1
  Administrative divisions No. of districts Total population  \
                    States           States           States   
0           Andhra Pradesh               26         49577103   
1        Arunachal Pradesh               26          1383727   
2                    Assam               35         31205576   
3                    Bihar               38        104099452   
4             Chhattisgarh               33         25545198   

  Population per district Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   States             States    

In [110]:
tableData[0]

,Districts of India,Districts of India.1
0,NaN,NaN
1,Category,second/third-level administrative division
2,Location,States and union territories of India
3,Number,797 (as of 20 November 2023)
4,Populations,"Greatest: Thane, Maharashtra—11,060,148 (2011 ..."
5,Areas,"Largest: Kutch, Gujarat—45,652 km2 (17,626 sq ..."
6,Densities,"Largest: Central Delhi, Delhi Smallest: Lower ..."
7,Government,District Administration
8,Subdivisions,"Revenue division (Sub-division) and,Tehsil (Ta..."


In [111]:
statesDf = tableData[1]
statesDf = statesDf.drop(28).drop(37)
statesDf

,Administrative divisions,No. of districts,Total population,Population per district,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0
,States,States,States,States,States,States,States,States,States
0,Andhra Pradesh,26,49577103,1906812,NaN,NaN,NaN,NaN,NaN
1,Arunachal Pradesh,26,1383727,53220,NaN,NaN,NaN,NaN,NaN
2,Assam,35,31205576,891588,NaN,NaN,NaN,NaN,NaN
3,Bihar,38,104099452,2739459,NaN,NaN,NaN,NaN,NaN
4,Chhattisgarh,33,25545198,774097,NaN,NaN,NaN,NaN,NaN
5,Goa,2,1458545,729273,NaN,NaN,NaN,NaN,NaN
6,Gujarat,33,60439692,1831506,NaN,NaN,NaN,NaN,NaN
7,Haryana,22,25351462,1152339,NaN,NaN,NaN,NaN,NaN
8,Himachal Pradesh,12,6864602,528046,NaN,NaN,NaN,NaN,NaN


In [112]:
states = sum(statesDf['Administrative divisions'].values.tolist(),[])
states

['Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chhattisgarh',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal',
 'Andaman and Nicobar Islands',
 'Chandigarh',
 'Dadra Nagar Haveli, Daman and Diu',
 'Jammu and Kashmir',
 'Ladakh',
 'Lakshadweep',
 'Delhi',
 'Puducherry']

In [113]:
[x for x in tableData[3].columns if 'Population' in x]

['Population (2021)']

In [132]:
stateNo = 3
for i in states:
    df = tableData[stateNo]
    colList = [x for x in df.columns if 'Population' in x]
    df = df.rename(columns={colList[0]: 'Population'})
    df['State/UnionTerritory']=i
    df['City Tier'] = np.nan
    df['Population'] = df['Population'].replace('–', 0).replace('-', 0)
    df['Population'] = pd.to_numeric(df['Population'])
    tier1Mask = df['Population'] >= 100000
    tier2Mask = df['Population'] <= 99999 & df['Population'] >= 50000
    tier3Mask = df['Population'] <= 49999
    stateNo+=1

In [133]:
tier1Mask

0     True
1    False
2     True
3    False
Name: Population, dtype: bool